# theano的dimshuffle
theano中的dimshuffle函数用于对张量的维度进行操作，可以增加维度，也可以交换维度，删除维度。  
以下规则总结自官方文档：    
* 'x'表示增加一维，从0d scalar到1d vector  
* (0, 1)表示一个与原先相同的2D向量  
* (1, 0)表示将2D向量的两维交换  
* (‘x’, 0) 表示将一个1d vector变为一个1xN矩阵  
* (0, ‘x’)将一个1d vector变为一个Nx1矩阵  
* (2, 0, 1) -> A x B x C to C x A x B （2表示第三维也就是C，0表示第一维A，1表示第二维B）  
* (0, ‘x’, 1) -> A x B to A x 1 x B 表示A，B顺序不变在中间增加一维  
* (1, ‘x’, 0) -> A x B to B x 1 x A 同理自己理解一下  
* (1,) -> 删除维度0，被移除的维度必须是可广播的（broadcastable），即(1xA to A)  

写了个小程序来验证一下  
首先定义了一个[0 1 2]的1D vector：v，v.dimshuffle(0)中的0表示第一维：3，也只有一维，所以不变。因为是1D的，所以shape只有（3，）

In [1]:
from __future__ import print_function
import theano
import numpy as np

v = theano.shared(np.arange(3))
# v.shape is a symbol expression, need theano.function or eval to compile it
v_disp = v.dimshuffle(0)
print('v.dimshuffle(0):',v_disp.eval())
print('v.dimshuffle(0).shape:',v_disp.shape.eval())

v.dimshuffle(0): [0 1 2]
v.dimshuffle(0).shape: [3]


v.dimshuffle('x',0)表示在第一维前加入一维，只要记住加了'x'就加了一维，所以大小变成了1x3

In [2]:
v_disp = v.dimshuffle('x', 0)
print("v.dimshuffle('x',0):",v_disp.eval())
print("v.dimshuffle('x',0).shape:",v_disp.shape.eval())

v.dimshuffle('x',0): [[0 1 2]]
v.dimshuffle('x',0).shape: [1 3]


在axis=0后插入一维，那么形状应该变为3 x 1

In [3]:
v_disp = v.dimshuffle(0,'x')
print("v.dimshuffle(0,'x'):",v_disp.eval())
print("v.dimshuffle(0,'x').shape:",v_disp.shape.eval())

v.dimshuffle(0,'x'): [[0]
 [1]
 [2]]
v.dimshuffle(0,'x').shape: [3 1]


在axis=0后插入两个新的axis，那么形状应该变为3 x 1 x 1

In [4]:
v_disp = v.dimshuffle(0,'x','x')
print("v.dimshuffle(0,'x','x'):",v_disp.eval())
print("v.dimshuffle(0,'x','x').shape:",v_disp.shape.eval())

v.dimshuffle(0,'x','x'): [[[0]]

 [[1]]

 [[2]]]
v.dimshuffle(0,'x','x').shape: [3 1 1]


在axis=0前后各插入一个新的axis，那么形状应该变为1 x 3 x 1

In [5]:
v_disp = v.dimshuffle('x',0,'x')
print("v.dimshuffle('x',0,'x'):",v_disp.eval())
print("v.dimshuffle('x',0,'x').shape:",v_disp.shape.eval())

v.dimshuffle('x',0,'x'): [[[0]
  [1]
  [2]]]
v.dimshuffle('x',0,'x').shape: [1 3 1]


在axis=0前插入两个新的axis，那么形状应该变为1 x 1 x 3

In [6]:
v_disp = v.dimshuffle('x','x',0)
print("v.dimshuffle('x','x',0):",v_disp.eval())
print("v.dimshuffle('x','x',0).shape:",v_disp.shape.eval())

v.dimshuffle('x','x',0): [[[0 1 2]]]
v.dimshuffle('x','x',0).shape: [1 1 3]


第二个例子，m是一个2x3矩阵

In [7]:
m = theano.shared(np.arange(6).reshape(2,3))
print("m:",m.eval())
print("m.shape:",m.shape.eval())

m: [[0 1 2]
 [3 4 5]]
m.shape: [2 3]


先确定0,'x',1的维数,0对应第一维（2），1表示第二维（3），'x'表示新加入的维度（1），所以结果维度是2x1x3
加括号的顺序按照从左到右（外->内)：  
1.先加最内层3，3表示括号内有3个数，因此是[0 1 2]和[3 4 5]  
2.再加中间层1，1表示括号内只有一个匹配的"[]"，因此是[[0 1 2]],[[3 4 5]]  
3.最后加最外层2，2表示括号内有两个匹配的"[]"（只算最外层的匹配），于是最后结果是  
```py
[[[0 1 2]]
 [[3 4 5]]]
```

In [8]:
m_disp = m.dimshuffle(0,'x',1)
print("m.dimshuffle(0,'x',1):",m_disp.eval())
print("m.dimshuffle(0,'x',1).shape:",m_disp.shape.eval())

m.dimshuffle(0,'x',1): [[[0 1 2]]

 [[3 4 5]]]
m.dimshuffle(0,'x',1).shape: [2 1 3]


同理，结果应该是1 x 2 x 3的张量

In [9]:
m_disp = m.dimshuffle('x', 0, 1)
print("m.dimshuffle('x',0,1):",m_disp.eval())
print("m.dimshuffle('x',0,1).shape:",m_disp.shape.eval())

m.dimshuffle('x',0,1): [[[0 1 2]
  [3 4 5]]]
m.dimshuffle('x',0,1).shape: [1 2 3]


同理，结果应该是2 x 3 x 1的张量

In [10]:
m_disp = m.dimshuffle(0, 1, 'x')
print("m.dimshuffle(0,1,'x'):",m_disp.eval())
print("m.dimshuffle(0,1,'x').shape:",m_disp.shape.eval())

m.dimshuffle(0,1,'x'): [[[0]
  [1]
  [2]]

 [[3]
  [4]
  [5]]]
m.dimshuffle(0,1,'x').shape: [2 3 1]


`1,'x',0`表示将axis0和axis1进行了交换，并在中间插入一个新的axis，因此结果应该是3 x 1 x 2

In [11]:
# amount to transpose
m_disp = m.dimshuffle(1,'x',0)
print("m.dimshuffle(1,'x',0):",m_disp.eval())
print("m.dimshuffle(1,'x',0).shape:",m_disp.shape.eval())

m.dimshuffle(1,'x',0): [[[0 3]]

 [[1 4]]

 [[2 5]]]
m.dimshuffle(1,'x',0).shape: [3 1 2]


(1,)表示删除一个第一维（axis=0），在初始化shared时要保证被删除的维度是可广播的  

In [12]:
m = theano.shared(np.array([[1,2,3]]),broadcastable=(True,False))
print(m.shape.eval())
print(m.dimshuffle(1,).shape.eval())

[1 3]
[3]
